# MiniProyecto 4: Optimización de parámetros

¡Bienvenidos al cuarto (mini)proyecto de la carrera de Data Science de Acamica! 

En este proyecto vamos a seguir trabajando (por última vez) con el dataset de propiedades en venta publicadas en el portal [Properati](www.properati.com.ar). El objetivo en este caso es optimizar los parámetros de los algoritmos que usamos en el proyecto pasado.

El dataset es el mismo del proyecto 3. Recordemos que las columnas que se agregan son:

* `barrios_match`: si coincide el barrio publicado con el geográfico vale 1, si no 0.

* `PH`, `apartment`, `house`: variables binarias que indican el tipo de propiedad.

* dummies de barrios: variables binarias con 1 o 0 según el barrio.

La métrica que vamos a usar para medir es RMSE (raíz del error cuadréatico medio), cuya fórmula es:

$$RMSE = \sqrt{\frac{\sum_{t=1}^n (\hat y_t - y_t)^2}{n}}$$

## Pandas - Levantamos el dataset

In [1]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
path_dataset = 'dataset/datos_properati_limpios_model.csv'
df = pd.read_csv(path_dataset)

**Separá** el dataset en entrenamiento (80%) y test (20%) utilizando como target la columna `price_aprox_usd`

In [2]:
# Hacé la separación en esta celda
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ParameterGrid


In [3]:
# Esto es lo que hace
np.random.seed(123)
X = df.drop(['price_aprox_usd'], axis=1)
y = df['price_aprox_usd']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

print(X_train.shape[0], X_test.shape[0])

5100 1276


## Scikit-learn - Entrenamiento

Para repasar los parámetros de árboles de decisión en Scikit-learn: 

http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html

En primer lugar veamos como hacer cross validation. Para eso necesitamos definir la cantidad de folds, en este caso vamos a usar 5.

GridSearchCV nos permite testear a través de un espacio de búsqueda de parámetros la mejor combinación posible dado un estimador.

Por ejemplo, en este caso probamos la profundidad máxima y la máxima cantidad de features para hacer los split. Ambos entre 1 y 5.
Recordemos que para hacer la optimización scikit-learn usa la métrica `neg_mean_squared_error` en lugar de `mean_squared_error`.

**Creá** una variable `param_grid` con valores del 1 al 5 para los atributos `max_depth` y `max_features`. 

In [4]:
# Creá en esta celda la variable param_grid
# Param grid puede ser una lista de mapas, mas abajo te das cuenta de por qué
param_grid = {"max_depth":[1,2,3,4,5],
              "max_features":[1,2,3,4,5]}

param_grid

{'max_depth': [1, 2, 3, 4, 5], 'max_features': [1, 2, 3, 4, 5]}

**Importá** `GridSearchCV` y `DecisionTreeRegressor`.

**Creá** una variable `grid_search` y asignale un `GridSearchCV` que recorra el `param_grid` que creaste con el algoritmos `DecisionTreeRegressor` y el un scoring de `neg_mean_squared_error`

In [5]:
# Importa y crea un GridSearchCV en esta celda

from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

regtree = DecisionTreeRegressor()
# El verbose y n_jobs al ser algo pequeño no hacen falta.
grid_search = GridSearchCV(regtree ,param_grid,'neg_mean_squared_error',cv = 5, verbose = 3,n_jobs = -1)

A continuación, realizá el `fit` del `grid_search` con el conjunto de entrenamiento

In [6]:
# Hace el fit de grid search en esta celda
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 125 out of 125 | elapsed:    1.8s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=DecisionTreeRegressor(criterion='mse', max_depth=None,
                                             max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             presort=False, random_state=None,
                                             splitter='best'),
             iid='warn', n_jobs=-1,
             param_grid={'max_depth': [1, 2, 3, 4, 5],
                         'max_features': [1, 2, 3, 4, 5]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='ne

Revisemos los resultados. Recordemos que no están expresados en RMSE.

In [ ]:
#grid_search.scorer_ <-- falta eso

In [18]:
results = grid_search.cv_results_
#results

**Mostrá** los `grid_scores` obtenidos durante el `grid_search`

In [8]:
# Mostrá los grid_scores en esta celda
grid_search.best_params_

{'max_depth': 5, 'max_features': 5}

De esta manera, el valor con mejor resultado (dado el espacio de búsqueda definido) es `max_depth` 3 y `max_features` 3.

**Mostrá** el mejor score y los mejores parámetros encontrados por `grid_search`

In [9]:
# Podrias mejorar esta celda

# print('Los mejores parametros son: {}, el mejor score es: {}'.format(grid_search.best_params_,
# grid_search.best_score_))
# Mostrás los resultados en esta celda
b_score= grid_search.best_score_
b_score

-742715589.3664114

Convertimos a RMSE.

In [10]:
def nmsq2rmse(score):
    return np.round(np.sqrt(-score), 2)

In [11]:
nmsq2rmse(b_score)

27252.81

__Encontrar el mejor modelo para el espacio de búsqueda dado__

* `"min_samples_split": [2, 10, 20]`
* `"max_depth": [None, 2, 5, 10, 15]`
* `"min_samples_leaf": [1, 5, 10, 15]`
* `"max_leaf_nodes": [None, 5, 10, 20]`

In [ ]:
# Acá se vuelve hacer un "param_grid" con todos los valores de la celda de arriba

Recordemos que `GridSearchCV` tiene como parámetro default `refit=True`. Esto significa que luego de hacer la corrida se ajusta el mejor modelo al conjunto de datos de entrada. De esta manera, se puede predecir directamente usando `best_estimator_`.

In [ ]:
# Acá se hacer el gridSearch con el param_grid de arriba

# EHHH tratá de acomodar el notebook para que sea un poco mas ordenado

In [12]:
optimised_decision_tree = grid_search.best_estimator_

__Evaluemos en testing el desempeño de este modelo.__

Como venimos trabajando, el resultado en testing será la medición que usaremos como benchmark para comparar este modelos con otros en el futuro, puesto que no estuvo en contacto con el dataset de test para la calibración.

In [13]:
from sklearn.metrics import mean_squared_error
y_opt_pred = optimised_decision_tree.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_opt_pred))
np.round(rmse)

29894.0

Vemos los primeros 10 resultados de la predicción del valor de propiedades.

In [14]:
val_real = pd.Series(y_test.values)
val_pred = pd.Series(y_opt_pred)

In [15]:
predicciones = pd.concat([val_real.rename('Valor real'),val_pred.rename('Valor Pred') ,abs(val_real-val_pred).rename('Dif(+/-)')] ,  axis=1)

In [16]:
predicciones.head(10)

,Valor real,Valor Pred,Dif(+/-)
0,80000.000,120134.144,40134.144
1,128000.000,120134.144,7865.856
2,150000.000,120134.144,29865.856
3,85000.000,120134.144,35134.144
4,135000.000,120134.144,14865.856
5,135000.000,139168.212,4168.212
6,68000.000,120134.144,52134.144
7,110000.000,148456.207,38456.207
8,134000.000,120134.144,13865.856
9,110000.000,120134.144,10134.144


In [31]:
param_grid ={ "min_samples_split": [2, 10, 20],
"max_depth": [None, 2, 5, 10, 15],
"min_samples_leaf": [1, 5, 10, 15],
"max_leaf_nodes": [None, 5, 10, 20]}

param_grid

{'min_samples_split': [2, 10, 20],
 'max_depth': [None, 2, 5, 10, 15],
 'min_samples_leaf': [1, 5, 10, 15],
 'max_leaf_nodes': [None, 5, 10, 20]}

In [32]:
regtree = DecisionTreeRegressor()

grid_search = GridSearchCV(regtree ,param_grid,'neg_mean_squared_error',cv = 5, verbose = 3,n_jobs = -1)

In [33]:
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 240 candidates, totalling 1200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 604 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done 1200 out of 1200 | elapsed:    9.0s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=DecisionTreeRegressor(criterion='mse', max_depth=None,
                                             max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             presort=False, random_state=None,
                                             splitter='best'),
             iid='warn', n_jobs=-1,
             param_grid={'max_depth': [None, 2, 5, 10, 15],
                         'max_leaf_nodes': [None, 5, 10, 20],
                         'min_samples_leaf': [1, 5, 10, 15],
                         'min_s

In [44]:
results = grid_search.cv_results_
#results

In [35]:
grid_search.best_params_

{'max_depth': 10,
 'max_leaf_nodes': None,
 'min_samples_leaf': 15,
 'min_samples_split': 2}

In [36]:
# Mostrás los resultados en esta celda
b_score= grid_search.best_score_
b_score

-459407773.74392384

In [37]:
nmsq2rmse(b_score)

21433.8

In [38]:
optimised_decision_tree = grid_search.best_estimator_

In [39]:
y_opt_pred = optimised_decision_tree.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_opt_pred))
np.round(rmse)

21301.0

In [40]:
val_real = pd.Series(y_test.values)
val_pred = pd.Series(y_opt_pred)

In [41]:
predicciones = pd.concat([val_real.rename('Valor real'),val_pred.rename('Valor Pred') ,abs(val_real-val_pred).rename('Dif(+/-)')] ,  axis=1)

In [42]:
predicciones.head(10)

,Valor real,Valor Pred,Dif(+/-)
0,80000.000,103438.660,23438.660
1,128000.000,135705.882,7705.882
2,150000.000,156075.759,6075.759
3,85000.000,102400.991,17400.991
4,135000.000,135571.622,571.622
5,135000.000,109560.000,25440.000
6,68000.000,75181.250,7181.250
7,110000.000,140444.444,30444.444
8,134000.000,158431.250,24431.250
9,110000.000,76701.202,33298.798
